In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points
from scipy import stats

#Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Helpful ML library
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler

#ML libraries
from sklearn.linear_model import LinearRegression

# Ignoring the errors
import warnings
warnings.filterwarnings('ignore')

# 1. Importing Data


In [ ]:
train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

## 1.2 Creating Categorical and Numerical features

In [ ]:
cat_features = []
num_features = []
for col in train.columns :
    if train[col].dtypes == 'object':
        cat_features.append(col )
    else:
        num_features.append(col)


train_ID = train['ID']
test_ID = test['ID']
train.drop('ID',axis=1, inplace = True)
test.drop('ID',axis=1, inplace = True)


## 1.3 Skeness and Kurtosis in SalePrice 

In [ ]:
sns.distplot(train['SalePrice'])
print('Skewness %0.02f' %train['SalePrice'].skew())
print('Kurtosis %0.02f' % train['SalePrice'].kurt())

from scipy import stats
from scipy.stats import norm, skew #for some statistics

# Plot histogram and probability
fig = plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
sns.distplot(train['SalePrice'] , fit=norm);
(mu, sigma) = norm.fit(train['SalePrice'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')
plt.subplot(1,2,2)
res = stats.probplot(train['SalePrice'], plot=plt)
plt.suptitle('Before transformation')

# Apply transformation
train.SalePrice = np.log1p(train.SalePrice )
# New prediction
y_train = train.SalePrice.values
y_train_orig = train.SalePrice


# Plot histogram and probability after transformation
fig = plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
sns.distplot(train['SalePrice'] , fit=norm);
(mu, sigma) = norm.fit(train['SalePrice'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')
plt.subplot(1,2,2)
res = stats.probplot(train['SalePrice'], plot=plt)
plt.suptitle('After transformation')

In [ ]:
#fig,ax = plt.subplots(2,2,figsize=(15,10))
fig = plt.figure(figsize=(15,5))
from scipy.stats import norm


plt.subplot(1,2,1)
sns.distplot(train['SalePrice'], fit=norm)
(mu, sigma) = norm.fit(train['SalePrice'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')

plt.subplot(1,2,2)
res = stats.probplot(train['SalePrice'], plot=plt)
plt.suptitle('Before transformation')

In [ ]:
#fig,ax = plt.subplots(2,2,figsize=(15,10))
train['SalePrice'] = np.log(train['SalePrice'])

fig = plt.figure(figsize=(15,5))


plt.subplot(1,2,1)
sns.distplot(train['SalePrice'], fit=norm)
(mu, sigma) = norm.fit(train['SalePrice'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')


plt.subplot(1,2,2)
res = stats.probplot(train['SalePrice'], plot=plt)
plt.suptitle('After transformation')

In [ ]:
data_features =pd.concat((train,test)).reset_index(drop=True)
print(data_features.shape)


## Missing Data

In [ ]:
data_features_na = data_features.isnull().sum()
data_features_na = data_features_na[data_features_na>0]
data_features_na.sort_values(ascending=False)

In [ ]:
total = data_features.isnull().sum().sort_values(ascending=False)
percent = (data_features.isnull().sum()/data_features.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

In [ ]:
data_features_na.index

In [ ]:
cat_features[1][0]

In [ ]:
fig, axx= plt.subplots(14,3,figsize=(20,50))
for n in range(0,14):
    for q in range(0,3):
        o=3*n+q
        if o<44 :
            sns.boxplot(x=cat_features[o],y='SalePrice',data=train,ax=axx[n][q])
            
            
plt.tight_layout()

In [ ]:
sns.distplot(train['BsmtFinSF1'])

In [ ]:
train_test = [train,test]
for col in train_test:
    col['GarageFinish'].fillna(col['GarageFinish'].mode()[0],inplace=True)    
    col['GarageCond'].fillna(col['GarageCond'].mode()[0],inplace=True)   
    col['LotFrontage'].fillna(col['LotFrontage'].median(),inplace=True)
    col['GarageQual'].fillna(col['GarageQual'].mode()[0],inplace=True)   
    col['FireplaceQu'].fillna(col['FireplaceQu'].mode()[0],inplace=True)   
    col['GarageYrBlt'].fillna(col['GarageYrBlt'].mode()[0],inplace=True)   
    col['GarageType'].fillna(col['GarageType'].mode()[0],inplace=True)   
    col['BsmtExposure'].fillna(col['BsmtExposure'].mode()[0],inplace=True)   
    col['BsmtCond'].fillna(col['BsmtCond'].mode()[0],inplace=True)   
    col['BsmtQual'].fillna(col['BsmtQual'].mode()[0],inplace=True)   
    col['BsmtFinType2'].fillna(col['BsmtFinType2'].mode()[0],inplace=True)   
    col['BsmtFinType1'].fillna(col['BsmtFinType1'].mode()[0],inplace=True)   
    col['MasVnrType'].fillna(col['MasVnrType'].mode()[0],inplace=True)   
    col['BsmtFullBath'].fillna(col['BsmtFullBath'].mode()[0],inplace=True)   
    col['BsmtHalfBath'].fillna(col['BsmtHalfBath'].mode()[0],inplace=True)   

    col['Utilities'].fillna(col['Utilities'].mode()[0],inplace=True)   

    col['Utilities'].fillna(col['Utilities'].mode()[0],inplace=True)   

    
    col['BsmtFinSF1'].fillna(col['BsmtFinSF1'].median(),inplace=True)
    col['BsmtFinSF2'].fillna(col['BsmtFinSF2'].median(),inplace=True)
    
    col['Exterior1st'].fillna(col['Exterior1st'].mode()[0],inplace=True)   

    col['Exterior2nd'].fillna(col['Exterior2nd'].mode()[0],inplace=True)   
    col['Electrical'].fillna(col['Electrical'].mode()[0],inplace=True)   
    col['Alley'].fillna(col['Alley'].mode()[0],inplace=True)   
    col['Fence'].fillna(col['Fence'].mode()[0],inplace=True)   
    col['Functional'].fillna(col['Functional'].mode()[0],inplace=True)   
    col['GarageCars'].fillna(col['GarageCars'].mode()[0],inplace=True)   
    col['KitchenQual'].fillna(col['KitchenQual'].mode()[0],inplace=True)   
    col['MiscFeature'].fillna(col['MiscFeature'].mode()[0],inplace=True)   
    col['SaleType'].fillna(col['SaleType'].mode()[0],inplace=True)   
    col['MSZoning'].fillna(col['MSZoning'].mode()[0],inplace=True) 
    
    col['GarageYrBlt'].fillna(col['GarageYrBlt'].mode()[0],inplace=True) 


    
    

    col['PoolQC'].fillna(col['PoolQC'].mode()[0],inplace=True)   

    col['BsmtUnfSF'].fillna(col['BsmtUnfSF'].median(),inplace=True)
    col['GarageArea'].fillna(col['GarageArea'].median(),inplace=True)
    col['MasVnrArea'].fillna(col['MasVnrArea'].median(),inplace=True)
    col['TotalBsmtSF'].fillna(col['TotalBsmtSF'].median(),inplace=True)
    col['LotFrontage'].fillna(col['LotFrontage'].median(),inplace=True)

    
train['SalePrice'].fillna(train['SalePrice'].median(),inplace=True)

In [ ]:
fig, axx= plt.subplots(12,3,figsize=(25,50))
print(num_features)
for n in range(0,12):
    for q in range(0,3):
        o=3*n+q
        if o<38 :
            sns.distplot(train[num_features[o]],ax=axx[n][q],fit=norm,label=num_features[o],rug=True)
            #axx[n][q].set_title(num_features[o],fontsize=11)
            
plt.tight_layout()


In [ ]:
    # Differentiate numerical features (minus the target) and categorical features
categorical_features = data_features.select_dtypes(include=['object']).columns
print(categorical_features)
numerical_features = data_features.select_dtypes(exclude = ["object"]).columns
print(numerical_features)

print("Numerical features : " + str(len(numerical_features)))
print("Categorical features : " + str(len(categorical_features)))
feat_num = data_features[numerical_features]
feat_cat = data_features[categorical_features]

In [ ]:
# Plot skew value for each numerical value
from scipy.stats import skew 
skewness = feat_num.apply(lambda x: skew(x))
skewness.sort_values(ascending=False)

skewness = skewness[abs(skewness) > 0.5]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))
print("Mean skewnees: {}".format(np.mean(skewness)))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    feat_num[feat] = boxcox1p(feat_num[feat], boxcox_normmax(feat_num[feat] + 1))
    data_features[feat] = boxcox1p(data_features[feat], boxcox_normmax(data_features[feat] + 1))
    
    
from scipy.stats import skew 
skewness.sort_values(ascending=False)

In [ ]:
for col in train_test:
    col['haspool'] = col['PoolArea'].apply(lambda x: 1 if x>0 else 0)
    col['hasgarage'] = col['GarageArea'].apply(lambda x: 1 if x>0 else 0)
    col['hasbasement'] = col['TotalBsmtSF'].apply(lambda x: 1 if x>0 else 0)
    col['hasfireplace'] = col['Fireplaces'].apply(lambda x: 1 if x>0 else 0)
    col.drop(['MasVnrArea', 'OpenPorchSF', 'WoodDeckSF', 'BsmtFinSF1','2ndFlrSF'], axis=1)

    


In [ ]:
vars = test.columns
# vars = numerical_features
figures_per_time = 4
count = 0 
y = train['SalePrice']
for var in vars:
    x = train[var]
#     print(y.shape,x.shape)
    plt.figure(count//figures_per_time,figsize=(25,5))
    plt.subplot(1,figures_per_time,np.mod(count,4)+1)
    plt.scatter(x, y,alpha=0.6,c=train['hasgarage']+1);
    plt.title('f model: T= {}'.format(var))
    count+=1

In [ ]:
vars_box = categorical_features
for var in vars_box:
    data = pd.concat([train['SalePrice'], train[var]], axis=1)
    f, ax = plt.subplots(figsize=(8, 6))
    fig = sns.boxplot(x=var, y="SalePrice", data=data)

In [ ]:
# Complete numerical correlation matrix
corrmat = train.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=1, square=True);

In [ ]:
# saleprice correlation matrix
corr_num = 15 #number of variables for heatmap
cols_corr = corrmat.nlargest(corr_num, 'SalePrice')['SalePrice'].index
corr_mat_sales = np.corrcoef(train[cols_corr].values.T)
sns.set(font_scale=1.25)
f, ax = plt.subplots(figsize=(12, 9))
hm = sns.heatmap(corr_mat_sales, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 7}, yticklabels=cols_corr.values, xticklabels=cols_corr.values)
plt.show()

In [ ]:
# pair plots for variables with largest correlation
var_num = 8
vars = cols_corr[0:var_num]

sns.set()
sns.pairplot(train[vars], size = 2.5)
plt.show();

In [ ]:
data_features = pd.concat((train, test)).reset_index(drop=True)


In [ ]:
y_train=train['SalePrice']
data_features = data_features.drop("SalePrice", axis = 1)
final_features = pd.get_dummies(data_features)

print(final_features.shape)
X = final_features.iloc[:len(y), :]
X_test = final_features.iloc[len(y):, :]
X.shape, y_train.shape, X_test.shape


print(X.shape,y_train.shape,X_test.shape)


In [ ]:
# Removes colums where the threshold of zero's is (> 99.95), means has only zero values 
overfit = []
for i in X.columns:
    counts = X[i].value_counts()
    zeros = counts.iloc[0]
    if zeros / len(X) * 100 > 99.95:
        overfit.append(i)

overfit = list(overfit)
overfit.append('MSZoning_C (all)')

X = X.drop(overfit, axis=1).copy()
X_test = X_test.drop(overfit, axis=1).copy()

print(X.shape,y_train.shape,X_test.shape)

In [ ]:
from datetime import datetime
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error , make_scorer
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from mlxtend.regressor import StackingCVRegressor
from sklearn.linear_model import LinearRegression

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [ ]:
kfolds = KFold(n_splits=18, shuffle=True, random_state=42)

# model scoring and validation function
def cv_rmse(model, X=X):
    rmse = np.sqrt(-cross_val_score(model, X, y,scoring="neg_mean_squared_error",cv=kfolds))
    return (rmse)

# rmsle scoring function
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [ ]:
lightgbm = LGBMRegressor(objective='regression', 
                                       num_leaves=4, #was 3
                                       learning_rate=0.01, 
                                       n_estimators=9000, #8000
                                       max_bin=200, 
                                       bagging_fraction=0.75,
                                       bagging_freq=5, 
                                       bagging_seed=7,
                                       feature_fraction=0.2, # 'was 0.2'
                                       feature_fraction_seed=7,
                                       verbose=-1,
                                       )

# xgboost = XGBRegressor(learning_rate=0.01,n_estimators=3460,
#                                      max_depth=3, min_child_weight=0,
#                                      gamma=0, subsample=0.7,
#                                      colsample_bytree=0.7,
#                                      objective='reg:linear', nthread=-1,
#                                      scale_pos_weight=1, seed=27,
#                                      reg_alpha=0.00006)



# setup models hyperparameters using a pipline
# The purpose of the pipeline is to assemble several steps that can be cross-validated together, while setting different parameters.
# This is a range of values that the model considers each time in runs a CV
e_alphas = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]
e_l1ratio = [0.8, 0.85, 0.9, 0.95, 0.99, 1]
alphas_alt = [14.5, 14.6, 14.7, 14.8, 14.9, 15, 15.1, 15.2, 15.3, 15.4, 15.5]
alphas2 = [5e-05, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008]




# Kernel Ridge Regression : made robust to outliers
ridge = make_pipeline(RobustScaler(), RidgeCV(alphas=alphas_alt, cv=kfolds))

# LASSO Regression : made robust to outliers
lasso = make_pipeline(RobustScaler(), LassoCV(max_iter=1e7, 
                    alphas=alphas2,random_state=42, cv=kfolds))

# Elastic Net Regression : made robust to outliers
elasticnet = make_pipeline(RobustScaler(), ElasticNetCV(max_iter=1e7, 
                         alphas=e_alphas, cv=kfolds, l1_ratio=e_l1ratio))


stack_gen = StackingCVRegressor(regressors=(ridge, lasso, elasticnet, lightgbm),
                                meta_regressor=elasticnet,
                                use_features_in_secondary=True)


svr = make_pipeline(RobustScaler(), SVR(C= 20, epsilon= 0.008, gamma=0.0003))

# store models, scores and prediction values 
models = {'Ridge': ridge,
          'Lasso': lasso, 
          'ElasticNet': elasticnet,
          'lightgbm': lightgbm,
          'Svd': svr}
#           'xgboost': xgboost}
predictions = {}
scores = {}

In [ ]:
for name, model in models.items():
    
    model.fit(X, y)
    predictions[name] = np.expm1(model.predict(X))
    
    score = cv_rmse(model, X=X)
    scores[name] = (score.mean(), score.std())

In [ ]:

print('---- Score with CV_RMSLE-----')
score = cv_rmse(ridge)
print("Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = cv_rmse(lasso)
print("Lasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = cv_rmse(elasticnet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = cv_rmse(lightgbm)
print("lightgbm score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = cv_rmse(svr)
print("SVR: {:.4f} ({:.4f})".format(score.mean(), score.std()))
# scores['svr'] = (score.mean(), score.std())

# score = cv_rmse(xgboost)
# print("xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


#Fit the training data X, y
print('----START Fit----',datetime.now())
print('Elasticnet')
elastic_model = elasticnet.fit(X, y)
print('Lasso')
lasso_model = lasso.fit(X, y)
print('Ridge')
ridge_model = ridge.fit(X, y)
print('lightgbm')
lgb_model_full_data = lightgbm.fit(X, y)
print('Svr')
svr_model_full_data = svr.fit(X, y)

# print('xgboost')
# xgb_model_full_data = xgboost.fit(X, y)


print('stack_gen')
stack_gen_model = stack_gen.fit(np.array(X), np.array(y))
print('----START Fit----',datetime.now())


In [ ]:
def blend_models_predict(X):
    return ((0.16  * elastic_model.predict(X)) + \
            (0.16 * lasso_model.predict(X)) + \
            (0.11 * ridge_model.predict(X)) + \
            (0.2 * lgb_model_full_data.predict(X)) + \
            (0.1 * svr_model_full_data.predict(X)) + \
#             (0.1 * xgb_model_full_data.predict(X)) + \
            (0.27 * stack_gen_model.predict(np.array(X))))

In [ ]:
print('RMSLE score on train data:')
print(rmsle(y, blend_models_predict(X)))

In [ ]:
print('Predict submission')
submission = pd.read_csv("../input/house-prices-advanced-regression-techniques/sample_submission.csv")
submission.iloc[:,1] = (np.expm1(blend_models_predict(X_test)))

In [ ]:
# q1 = submission['SalePrice'].quantile(0.0042)
# q2 = submission['SalePrice'].quantile(0.99)
# # Quantiles helping us get some extreme values for extremely low or high values 
# submission['SalePrice'] = submission['SalePrice'].apply(lambda x: x if x > q1 else x*0.77)
# submission['SalePrice'] = submission['SalePrice'].apply(lambda x: x if x < q2 else x*1.1)
submission.to_csv("submission.csv", index=False)